# comercio_ext_indices.tb_nomenclatura_brasileira
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NBM_delta`
## 📌 Descrição do arquivo

Tabela de referência com o **código NBM** e sua **descrição textual** do produto.

**NBM (Nomenclatura Brasileira de Mercadorias)** é uma classificação histórica de produtos usada no Brasil antes (ou em paralelo, dependendo do recorte temporal) da consolidação do uso da **NCM**.  
Neste projeto, ela serve como **tabela dimensão/legado** para compatibilização e histórico.

Tipo: Dimensão de produto (legado)  
Uso: Referência / bridge com NCM (via `NBM_NCM.csv`)

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_NBM`|Código NBM do produto|Identificador do produto na nomenclatura NBM|
|`NO_NBM`|Descrição do produto|Nome/descrição do item conforme NBM|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable
jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

In [0]:
bronzePath = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/nbm/"
silverPath = "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_nomenclatura_brasileira/"
silverTable = "silver_comercio_ext_indices.tb_nomenclatura_brasileira"

In [0]:
silverSchema = T.StructType([
    T.StructField("CO_NBM",   T.StringType(),   False),
    T.StructField("NO_NBM",   T.StringType(),   False),
    T.StructField("TS_REF",   T.TimestampType(),False),
    T.StructField("NM_ORIGEM",T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**

In [0]:
df_norm = (
    df_bronze
    .withColumn("CO_NBM", F.upper(F.trim(F.col("CO_NBM").cast(T.StringType()))))
    .withColumn("NO_NBM", F.col("NO_NBM").cast(T.StringType()))
    .withColumn("TS_REF", F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/comercio_ext_indices/NBM_delta"))
)

###Validações

In [0]:
df_valid = df_norm.filter(F.col("CO_NBM").isNotNull())



In [0]:
df_dedup = df_valid.dropDuplicates(["CO_NBM"])


In [0]:
df_silver = df_dedup.select("CO_NBM","NO_NBM","TS_REF","NM_ORIGEM")

##Carga
> #### **merge**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_indices.tb_nomenclatura_brasileira")

merge_condition = """
      t.CO_NBM = s.CO_NBM
    """

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "t.NO_NBM":   "s.NO_NBM",
        "t.TS_REF":     "s.TS_REF",
        "t.NM_ORIGEM":  "s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
    .execute()
)